Hello! This is a tutorial for a game similar to peek-a-boo which the Naos can play. The game is called "Where's the ball?" and it uses the idea of subscribing and ubsubscribing to events in order to work. 

First, we must import our time library from Python and import ALProxy, ALBroker, and ALModule from naoqi. These libraries include the APIs we will use. 

In [6]:
import time
from naoqi import ALProxy
from naoqi import ALBroker
from naoqi import ALModule

Now, I am going to create a Robot class and define a constructor. Inside the constructor I will initialize all the APIs I will use and set the broker to listen to any Nao IP addresses available. 

In [ ]:
class Robot(ALModule):
    def __init__(self, name):
        self._name = name
        self.broker = ALBroker("myBroker",
           "0.0.0.0",   # listen to anyone
           0,           # find a free port and use it
           self._name + ".brynmawr.edu",         # parent broker IP
           9559)       # parent broker port

        ALModule.__init__(self, name)
        
        #Initializes APIs I will use
        self.tts = ALProxy("ALTextToSpeech")
        self.posture = ALProxy("ALRobotPosture")
        self.motion = ALProxy("ALMotion")
        self.memory = ALProxy("ALMemory")
        self.awareness = ALProxy("ALBasicAwareness")

Then, I will create the method called cover_eyes() inside the constructor which forces the Nao to raise his hands and cover his eyes. This method uses the ALMotion API to set the angles for the ShoulderRoll, ElbowRoll, ElbowYaw, and ShoulderPitch joints. 

In [ ]:
        def cover_eyes(self):
            self.awareness.stopAwareness()
            self.motion.setStiffnesses("Head", 1.0)
            self.motion.setAngles("HeadPitch", 0, 0.1)
            self.motion.setAngles("HeadYaw", 0, 0.1)
            self.motion.setStiffnesses("Body", 1.0)
            self.motion.setAngles("RShoulderRoll", [-0.0614018440246582], 0.5)
            self.motion.setAngles("RElbowRoll", [1.2809319496154785], 0.5 )
            self.motion.setAngles("RElbowYaw", [0.5414600372314453], 0.5 )
            self.motion.setAngles("RShoulderPitch", [0.05364799499511719], 0.5 )

            self.motion.setAngles("LShoulderRoll", [0.0614018440246582], 0.5)
            self.motion.setAngles("LElbowRoll", [-1.2809319496154785], 0.5 )
            self.motion.setAngles("LElbowYaw", [-0.5414600372314453], 0.5 )
            self.motion.setAngles("LShoulderPitch", [-0.05364799499511719], 0.5 )

I will also create the method uncover_eyes() which tells the Nao to lower his hands to neutral sitting position.

In [ ]:
        def uncover_eyes(self):
            self.awareness.startAwareness()
            self.motion.setStiffnesses("Body", 1.0)
            self.motion.setAngles("RShoulderRoll", [-0.25928807258605957], 0.5)
            self.motion.setAngles("RElbowRoll", [1.245649814605713], 0.5 )
            self.motion.setAngles("RElbowYaw", [0.5015759468078613], 0.5 )
            self.motion.setAngles("RShoulderPitch", [0.964928150177002], 0.5 )

            self.motion.setAngles("LShoulderRoll", [0.25928807258605957], 0.5)
            self.motion.setAngles("LElbowRoll", [-1.245649814605713], 0.5 )
            self.motion.setAngles("LElbowYaw", [-0.5015759468078613], 0.5 )
            self.motion.setAngles("LShoulderPitch", [0.964928150177002], 0.5 )

Now, I will have the robot start off with a greeting and move to its sitting position if it's not already sitting down. 

In [ ]:
        self.tts.say("Hello, my name is "+ self._name + "Let's play a game called" "Where's the ball")
        
        #self.posture.goToPosture("StandInit", 0.5)
        uncover_eyes(self)
        currentPosture = self.posture.getPosture()
        
        if currentPosture != 'Sit':
            self.tts.say("First, place me in an open space so I can sit down.")
            time.sleep(5.0)
            self.posture.goToPosture("Sit", 0.5)
            time.sleep(3.0)
            self.tts.say("Thank you, now I feel great.")

Now, I will tell the robot to give instructions out loud on how to play the game. I will also start a timer so that the robot can cover his eyes for less than 10 seconds and then uncover them.

In [ ]:
        self.tts.say("Grab the red ball and hide it from me." + 
                     " I will tell you once I see the ball.")
        
        self.lastSeen = 0
        self.lastNotSeen = 0
        
        START = time.time()

        while time.time() - START < 10:

            cover_eyes(self)

            if time.time() - self.lastNotSeen > 5:
                self.tts.say("Where's the ball? I don't see the ball")
                self.lastNotSeen = time.time()    
        
        uncover_eyes(self)

Now, I will subscribe to the event called "redBallDetected" so that the robot can speak everytime he sees the red ball. 

In [ ]:
        self.memory.subscribeToEvent("redBallDetected",
            self._name,
            "onredBallDetected")

Below, I will define the method onredBallDetected(). For instance, the robot must be unsubscribed to the event "redBallDetected" when speaking. If we don't unsubscribe the robot from the event, it will create a queue of all the instances when it sees the ball and it will keep saying "I see the ball" even when the ball is no longer in his field of vision. Then, I will also start a timer so that the robot can only say "I see the ball" once every five seconds.

In [ ]:
def onredBallDetected(self, *args):
        # Unsubscribe to the event when talking,
        # to avoid repetitions

        self.memory.unsubscribeToEvent("redBallDetected",
            self._name)
        if time.time() - self.lastSeen > 5:
            self.tts.say("I see the ball")
            self.lastSeen = time.time()
            
        # Subscribe again to the event
        self.memory.subscribeToEvent("redBallDetected",
            self._name,
            "onredBallDetected")

I will also create a method called stop() so that I can unsubscribe to the event and make the program stop.

In [ ]:
    def stop(self):
        self.memory.unsubscribeToEvent("redBallDetected",
            self._name)
        self.broker.shutdown()

Now, I will create a new robot and execute the code. I will use Hedwig as an example. It is important to remember that my broker/name must be exactly the same as the global variable name.

In [11]:
 hedwig = Robot("hedwig")

Once I have ran the code above and proven know it works, I can execute the stop() method to force the broker to shut down.

In [ ]:
hedwig.stop()